In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 反馈或问题？

如果有任何反馈或问题，请打开一个[问题](https://github.com/googleapis/python-aiplatform/issues)。

Python的Vertex SDK：AutoML视频动作识别示例
要使用这个Jupyter笔记本，将笔记本复制到已安装Tensorflow的Google Cloud笔记本实例中并打开它。您可以运行每个步骤或单元，并查看其结果。要运行一个单元，使用Shift+Enter。Jupyter会自动显示每个单元最后一行的返回值。有关在Google Cloud笔记本中运行笔记本的更多信息，请参阅[Google Cloud笔记本指南](https://cloud.google.com/vertex-ai/docs/general/notebooks)。

本笔记本演示了如何使用Vertex AI视频数据集创建AutoML视频动作识别模型，以及如何为批量预测提供模型。您需要提供一个存储数据集的存储桶。

注意：在测试此SDK时，您可能会因培训、预测、存储或使用其他GCP产品而产生费用。

### 为 Python 安装 Vertex SDK


安装 SDK 后，内核将会自动重新启动。

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

###输入您的项目和GCS存储桶

在下面的单元格中输入您的项目ID。然后运行该单元格，以确保云SDK在此笔记本中的所有命令中使用正确的项目。

In [ ]:
MY_PROJECT = "YOUR PROJECT ID"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

In [ ]:
import sys

if "google.colab" in sys.modules:
    import os

    from google.colab import auth

    auth.authenticate_user()
    os.environ["GOOGLE_CLOUD_PROJECT"] = MY_PROJECT

### 设置您的任务名称和GCS前缀

如果您想要将所有输入和输出文件集中在GCS位置下。

In [ ]:
TASK_TYPE = "mbsdk_automl-video-training"
PREDICTION_TYPE = "action_recognition"
MODEL_TYPE = "CLOUD"

TASK_NAME = f"{TASK_TYPE}_{PREDICTION_TYPE}"
BUCKET_NAME = MY_STAGING_BUCKET.split("gs://")[1]
GCS_PREFIX = TASK_NAME

print(f"Bucket Name:    {BUCKET_NAME}")
print(f"Task Name:      {TASK_NAME}")

# HMDB：一个庞大的人体运动数据库
我们为演示准备了一些训练数据和预测数据，使用[HMDB数据集](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database)。

HMDB数据集根据知识共享署名4.0国际许可证许可。要查看此许可证的副本，请访问https://creativecommons.org/licenses/by/4.0/

有关此数据集的更多信息，请访问：https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/

In [ ]:
automl_video_demo_train_data = "gs://automl-video-demo-data/hmdb_golf_swing_all.csv"
automl_video_demo_batch_prediction_data = (
    "gs://automl-video-demo-data/hmdb_golf_swing_predict.jsonl"
)

### 复制 AutoML 视频演示训练数据以创建托管数据集

In [ ]:
gcs_source_train = f"gs://{BUCKET_NAME}/{TASK_NAME}/data/video_action_recognition.csv"

In [ ]:
!gsutil cp $automl_video_demo_train_data $gcs_source_train

使用托管视频数据集运行AutoML视频训练

## 初始化Python的Vertex SDK

为Vertex AI初始化*client*。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

## 在 Vertex AI 上创建数据集
现在我们将使用之前准备好的 csv 文件来创建一个 Vertex AI 视频数据集。请选择以下选项之一。

Option 1: 使用MBSDK VideoDataset 类

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name=f"temp-{TASK_NAME}",
    gcs_source=gcs_source_train,
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.action_recognition,
    sync=False,
)

选项2：使用MBSDK数据集类
```
dataset = aiplatform.Dataset.create(
    display_name=f'temp-{TASK_NAME}',
    metadata_schema_uri=aiplatform.schema.dataset.metadata.video,
    gcs_source=gcs_source_train, 
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.action_recognition,
    sync=False
)
```

In [ ]:
dataset.wait()

在Vertex AI上启动一个训练任务并创建一个模型。

###配置一个培训任务

In [ ]:
job = aiplatform.AutoMLVideoTrainingJob(
    display_name=f"temp-{TASK_NAME}",
    prediction_type=PREDICTION_TYPE,
    model_type=MODEL_TYPE,
)

运行训练任务

In [ ]:
model = job.run(
    dataset=dataset,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
    model_display_name=f"temp-{TASK_NAME}",
    sync=False,
)

In [ ]:
model.wait()

# 在模型上进行批量预测任务

### 复制 AutoML 视频演示预测数据，用于创建批量预测作业

In [ ]:
gcs_source_batch_prediction = f"gs://{BUCKET_NAME}/{TASK_NAME}/data/video_action_recognition_batch_prediction.jsonl"
gcs_destination_prefix_batch_prediction = (
    f"gs://{BUCKET_NAME}/{TASK_NAME}/batch_prediction"
)

In [ ]:
!gsutil cp $automl_video_demo_batch_prediction_data $gcs_source_batch_prediction

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"temp-{TASK_NAME}",
    gcs_source=gcs_source_batch_prediction,
    gcs_destination_prefix=gcs_destination_prefix_batch_prediction,
    sync=False,
)

In [ ]:
batch_predict_job.wait()
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

In [ ]:
import json

import tensorflow as tf

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            break

In [ ]:
line